In [120]:
import tensorflow
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
import tensorflow as tf

In [121]:
import pandas as pd
import sklearn
import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import metrics
import nltk
import string


from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
import re
from nltk.stem.porter import *

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lucyc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lucyc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [122]:
df = pd.read_csv('nuforc_reports.csv')

In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112095 entries, 0 to 112094
Data columns (total 12 columns):
summary           112059 non-null object
city              111851 non-null object
state             103722 non-null object
date_time         109726 non-null object
shape             108580 non-null object
duration          108382 non-null object
stats             112042 non-null object
report_link       112095 non-null object
text              112041 non-null object
posted            109726 non-null object
city_latitude     89359 non-null float64
city_longitude    89359 non-null float64
dtypes: float64(2), object(10)
memory usage: 10.3+ MB


In [124]:
df.isnull().sum()

summary              36
city                244
state              8373
date_time          2369
shape              3515
duration           3713
stats                53
report_link           0
text                 54
posted             2369
city_latitude     22736
city_longitude    22736
dtype: int64

In [125]:
df[['summary', 'text']].head()

,summary,text
0,"Three saucer shaped ships. High in the sky, m...","Three saucer shaped ships. High in the sky, m..."
1,Tear-drop shaped silent craft with sectional f...,Tear-drop shaped silent craft with sectional f...
2,Green orb shot out of white light ((NUFORC No...,Green orb shot out of white light I saw a whit...
3,Saw three circular orange lights traveling sou...,Saw three circular orange lights traveling sou...
4,Light moving slowly thru the sky towards the W...,Light moving slowly thru the sky towards the w...


In [126]:
df[df['text'].isnull()]

,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude
17151,Glowing fireball object crashed at Maverick's...,El Granada,CA,1995-01-01T02:00:00,fireball,45 seconds,NaN,http://www.nuforc.org/webreports/025/S25864.html,NaN,2002-11-04T00:00:00,37.502700,-122.469400
22560,Objects flickering red and green seemingly hov...,Toronto,ON,2003-08-16T22:30:00,unknown,1 hour 30. minutes +,NaN,http://www.nuforc.org/webreports/030/S30727.html,NaN,2003-08-28T00:00:00,43.675886,-79.414188
24968,Explosion in the sky,New Effington,SD,2004-02-14T20:17:00,unknown,2 seconds,NaN,http://www.nuforc.org/webreports/035/S35101.html,NaN,2004-03-02T00:00:00,45.862000,-96.948100
25235,chevron shaped craft hovering near our home,Monkland,ON,2004-02-19T19:30:00,chevron,30 Minutes,NaN,http://www.nuforc.org/webreports/035/S35193.html,NaN,2004-03-02T00:00:00,45.200100,-74.866000
25984,Bright greenish / white light that flew in a s...,Gatineau,QC,2003-11-26T21:41:00,light,3 Seconds,NaN,http://www.nuforc.org/webreports/033/S33439.html,NaN,2003-12-09T00:00:00,45.484218,-75.670763
27151,"three nights, in one week, glowing green and r...",Santa Fe,NM,2004-05-05T20:30:00,oval,1/2 hour,NaN,http://www.nuforc.org/webreports/036/S36882.html,NaN,2004-06-04T00:00:00,35.661083,-105.953907
27197,ufo at sunset,Seattle,WA,2004-05-12T19:15:00,light,15-20 sec,NaN,http://www.nuforc.org/webreports/036/S36880.html,NaN,2004-06-04T00:00:00,47.609046,-122.326028
27201,oval shape bright as a full moon,Jacksonville,FL,2004-05-13T00:20:00,oval,2 seconds,NaN,http://www.nuforc.org/webreports/036/S36883.html,NaN,2004-06-04T00:00:00,30.266905,-81.616373
27213,lots of UFOs,Perth (Western Australia),NaN,2004-05-13T23:00:00,cigar,50,NaN,http://www.nuforc.org/webreports/036/S36881.html,NaN,2004-06-04T00:00:00,NaN,NaN
27699,"Two separate ""fireball"" incidents in very dist...",Bullhead City,AZ,1987-06-15T18:30:00,fireball,?? 10min ??,NaN,http://www.nuforc.org/webreports/036/S36103.html,NaN,2004-04-27T00:00:00,35.084078,-114.562823


In [127]:
#replace missing text with summary
df['text']= df['text'].fillna(value = df['summary'])

In [128]:
#drop rows where there's no summary or text
reports = df.dropna(axis=0, how='all', subset=['text','summary'])

In [129]:
reports.isnull().sum()

summary              35
city                244
state              8373
date_time          2369
shape              3515
duration           3713
stats                53
report_link           0
text                  0
posted             2369
city_latitude     22736
city_longitude    22736
dtype: int64

In [130]:
#replace missing text with summary
reports['text'].fillna(value = reports['summary'], inplace=True)

C:\Users\Lucyc\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [131]:
reports['text'].isnull().sum()

0

In [132]:
reports.isnull().sum()

summary              35
city                244
state              8373
date_time          2369
shape              3515
duration           3713
stats                53
report_link           0
text                  0
posted             2369
city_latitude     22736
city_longitude    22736
dtype: int64

In [133]:
def remove_punctuation (text):
  
  new_text = "".join([char for char in text if char not in string.punctuation])
  return new_text

reports['clean_text'] = reports['text'].astype(str).apply(lambda x:remove_punctuation(x))


C:\Users\Lucyc\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [134]:
stopword = nltk.corpus.stopwords.words('english')
stopword

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [135]:
def tokenize (text):
  tokens = re.split('\W+', text)
  return tokens [1:]

reports['tokenized_text'] = reports['clean_text'].apply(lambda x: tokenize(x))


C:\Users\Lucyc\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [142]:
# # stopword = nltk.corpus.stopwords.words('english')

# # reports['nostop_text'] = reports['clean_text'].str.lower().str.split()  
# # reports['nostop_text'] = reports['nostop_text'] .apply(lambda x: [item for item in x if item not in stopword])
# # reports['nostop_text'] =[" ".join(text) for text in reports['nostop_text'] .values]

# stopword = nltk.corpus.stopwords.words('english')
# reports['nostop_text']=reports['clean_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stopword))

# reports.head()


stopword = nltk.corpus.stopwords.words('english')

def remove_stopwords(tokenized_text):
  text = [word for word in tokenized_text if word not in stopword]
  return text

reports['nostop_text']= reports['tokenized_text'].apply(lambda x:remove_stopwords(x))


C:\Users\Lucyc\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [143]:
# wn = WordNetLemmatizer()

# def lemmatizing(nostop_text):
#   text = [wn.lemmatize(word) for word in tokenized_text]
#   return text

# reports['lemmatized_text'] = reports['nostop_text'].apply(lambda x:lemmatizing(x))

In [144]:
shape_df = reports[pd.notnull(reports['shape'])]

In [145]:
shape_df

,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude,clean_text,tokenized_text,nostop_text
0,"Three saucer shaped ships. High in the sky, m...",Salem,OR,NaN,disk,15 minutes,Occurred : 8/15/1950 14:00 (Entered as : 08/1...,http://www.nuforc.org/webreports/135/S135871.html,"Three saucer shaped ships. High in the sky, m...",NaN,44.941247,-123.004235,Three saucer shaped ships High in the sky met...,"[saucer, shaped, ships, High, in, the, sky, me...","[saucer, shaped, ships, High, sky, metallic, s..."
1,Tear-drop shaped silent craft with sectional f...,Ellsworth,ME,NaN,teardrop,5 minutes,Occurred : 8/15/1967 21:30 (Entered as : 08/0...,http://www.nuforc.org/webreports/132/S132451.html,Tear-drop shaped silent craft with sectional f...,NaN,44.651300,-68.450700,Teardrop shaped silent craft with sectional fr...,"[shaped, silent, craft, with, sectional, front...","[shaped, silent, craft, sectional, front, end,..."
2,Green orb shot out of white light ((NUFORC No...,San Antonio,TX,2017-01-25T20:45:00,light,90 minutes,Occurred : 1/25/2017 20:45 (Entered as : 01/2...,http://www.nuforc.org/webreports/132/S132305.html,Green orb shot out of white light I saw a whit...,2017-01-26T00:00:00,29.488866,-98.475240,Green orb shot out of white light I saw a whit...,"[orb, shot, out, of, white, light, I, saw, a, ...","[orb, shot, white, light, I, saw, white, light..."
3,Saw three circular orange lights traveling sou...,Port Saint Lucie,FL,2017-02-24T20:45:00,circle,5 minutes,Occurred : 2/24/2017 20:45 (Entered as : 02/2...,http://www.nuforc.org/webreports/132/S132890.html,Saw three circular orange lights traveling sou...,2017-03-10T00:00:00,27.285686,-80.363444,Saw three circular orange lights traveling sou...,"[three, circular, orange, lights, traveling, s...","[three, circular, orange, lights, traveling, s..."
4,Light moving slowly thru the sky towards the W...,Whitefish,MT,2017-02-22T21:00:00,light,15 minutes,Occurred : 2/22/2017 21:00 (Entered as : 02/2...,http://www.nuforc.org/webreports/132/S132774.html,Light moving slowly thru the sky towards the w...,2017-03-10T00:00:00,48.411100,-114.337600,Light moving slowly thru the sky towards the w...,"[moving, slowly, thru, the, sky, towards, the,...","[moving, slowly, thru, sky, towards, west, Fir..."
5,Bright pulsating white light brighter than Ven...,Swanville,ME,2017-02-22T19:30:00,light,45 minutes,Occurred : 2/22/2017 19:30 (Entered as : 02/2...,http://www.nuforc.org/webreports/132/S132754.html,Bright pulsating white light brighter than Ven...,2017-02-22T00:00:00,NaN,NaN,Bright pulsating white light brighter than Ven...,"[pulsating, white, light, brighter, than, Venu...","[pulsating, white, light, brighter, Venus, vic..."
6,((HOAX??)) Loud boom heard.,Duxbury,MA,2017-01-31T06:00:00,unknown,2,Occurred : 1/31/2017 06:00 (Entered as : 01/3...,http://www.nuforc.org/webreports/132/S132401.html,Loud boom heard. ((NUFORC Note: Witness elect...,2017-02-03T00:00:00,42.049490,-70.693082,Loud boom heard NUFORC Note Witness elects to...,"[boom, heard, NUFORC, Note, Witness, elects, t...","[boom, heard, NUFORC, Note, Witness, elects, r..."
7,Two erratic bright white lights seen in Southe...,Studio City,CA,2017-01-29T10:34:00,light,3 minutes,Occurred : 1/29/2017 10:34 (Entered as : 01/2...,http://www.nuforc.org/webreports/132/S132369.html,Two erratic bright white lights seen in Southe...,2017-02-03T00:00:00,34.138265,-118.392057,Two erratic bright white lights seen in Southe...,"[erratic, bright, white, lights, seen, in, Sou...","[erratic, bright, white, lights, seen, Souther..."
8,There was only one UFO. It was yellow orange-...,Hernando,FL,2017-01-25T22:45:00,light,1 minute,Occurred : 1/25/2017 22:45 (Entered as : 1/25...,http://www.nuforc.org/webreports/132/S132302.html,There was only one UFO. It was yellow orange-...,2017-02-03T00:00:00,28.935600,-82.389500,There was only one UFO It was yellow orangeis...,"[was, only, one, UFO, It, was, yellow, orangei...","[

In [ ]:
# import tensorflow.keras
# from tensorflow import keras
# from keras.preprocessing.text import Tokenizer

# tokenizer = Tokenizer(num_words=200000)
# tokenizer.fit_on_texts(shape_df['nostop_text'].values)


In [ ]:
# tokenizer.word_index

In [ ]:
# len(tokenizer.word_index)

In [ ]:

# max_features = 20000 #refers to the number of unique words to be considered
# maxlen = 100  # cut texts after this number of words (among top max_features most common words)
# batch_size = 32
# x = tokenizer.texts_to_sequences(shape_df['nostop_text'].values)
# x= keras.preprocessing.sequence.pad_sequences(x, maxlen=maxlen)

In [146]:
shape_df.isnull().sum()

summary               8
city                141
state              7973
date_time          2262
shape                 0
duration           2757
stats                52
report_link           0
text                  0
posted             2262
city_latitude     21769
city_longitude    21769
clean_text            0
tokenized_text        0
nostop_text           0
dtype: int64

In [147]:
shape_df['shape'].value_counts()

light        22761
circle       11499
triangle     10488
fireball      8652
unknown       7879
other         7724
sphere        7362
disk          7005
oval          5033
formation     3456
changing      2715
cigar         2679
flash         1963
rectangle     1848
cylinder      1725
diamond       1650
chevron       1294
teardrop      1013
egg           1004
cone           468
cross          344
delta            8
round            2
crescent         2
hexagon          1
dome             1
flare            1
changed          1
pyramid          1
Name: shape, dtype: int64

In [148]:
new_shape_df = shape_df.replace({'shape' : {'disk': 'circle','unknown':'other','teardrop':'oval',
        'egg': 'oval','fireball': 'sphere', 'delta':'triangle', 'pyramid':'triangle', 'cigar':'cylinder',
       'round': 'sphere', 'changed':'changing', 'flare':'light'}})

In [149]:
new_shape_df['shape'].value_counts()

light        22762
circle       18504
sphere       16016
other        15603
triangle     10497
oval          7050
cylinder      4404
formation     3456
changing      2716
flash         1963
rectangle     1848
diamond       1650
chevron       1294
cone           468
cross          344
crescent         2
dome             1
hexagon          1
Name: shape, dtype: int64

In [150]:
texts=[" ".join(text) for text in new_shape_df['nostop_text'].values]

In [151]:
Encoder = LabelEncoder()
y= Encoder.fit_transform(new_shape_df['shape'])

In [152]:
num_classes = len(np.unique(y))

In [168]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(texts, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=1)

In [169]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# #TEXT REPRESENTATION
#word level tf-idf
Tfidf_vect = TfidfVectorizer(max_features=6000)

Tfidf_vect.fit(texts)
x_train_tfidf = Tfidf_vect.transform(x_train)
x_test_tfidf = Tfidf_vect.transform(x_test)


# ngram level tf-idf 
# tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
# tfidf_vect_ngram.fit(texts)
# X_train_tfidf_ngram =  tfidf_vect_ngram.transform(x_train)
# X_test_tfidf_ngram =  tfidf_vect_ngram.transform(x_test)

# # characters level tf-idf
# tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
# tfidf_vect_ngram_chars.fit(texts)
# X_train_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(x_train) 
# X_test_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(x_test) 


# from sklearn.feature_extraction.text import CountVectorizer

# count_vect = CountVectorizer(stop_words="english", analyzer='word', 
#                             ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=4)
# X_counts = count_vect.fit(texts)
# x_train_count =  count_vect.transform(x_train)
# x_test_count =  count_vect.transform(x_test)

In [ ]:
# print(dict(zip(Tfidf_vect.get_feature_names(), Tfidf_vect.idf_)))

In [ ]:
d = pd.DataFrame()
d['Word'] = Tfidf_vect.get_feature_names()

In [ ]:
d['idf'] = Tfidf_vect.idf_

In [ ]:
# d.sort_values(by=['idf'], ascending=False)

In [ ]:
# Tfidf_vect.vocabulary_

In [138]:
# # fit the training dataset on the NB classifier
# from sklearn import model_selection, naive_bayes, svm, metrics
# from sklearn import preprocessing
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score


# # Naive Bayes on Count Vectors
# def NaiveBayes(X_train, Y_train, X_test, Y_test, vectorizer):
#     Naive = naive_bayes.MultinomialNB()
#     Naive.fit(X_train,Y_train)
#     predictions_NB = Naive.predict(X_test)
#     probs = Naive.predict_proba(X_test)
#     probability = np.amax(probs, 1)
#     print(vectorizer,"Accuracy Score -> ",accuracy_score(predictions_NB, Y_test)*100)
#     print(metrics.classification_report(Y_test, predictions_NB))
#     summary = pd.DataFrame({'%':probability,'Prediction':predictions_NB})
#     print(summary)


# # NaiveBayes(x_train_count, y_train, x_test_count, y_test, 'Count')
# # NaiveBayes(x_train_tfidf, y_train, x_test_tfidf, y_test, 'Word')
# NaiveBayes(X_train_tfidf_ngram, y_train, X_test_tfidf_ngram, y_test, 'Ngram')
# # NaiveBayes(X_train_tfidf_ngram_chars, Y_train, X_test_tfidf_ngram_chars, Y_test, 'Ngram Char')




In [156]:
from sklearn.model_selection import GridSearchCV
naive_model = naive_bayes.MultinomialNB()
param_grid = {
    'alpha': [0.1, 0.5, 1, 1.5, 20]
}
# Instantiate the grid search model
nv_clf = GridSearchCV(estimator = naive_model, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2).fit(x_train_tfidf, y_train)

clf = nv_clf.best_estimator_

Fitting 3 folds for each of 5 candidates, totalling 15 fits


C:\Users\Lucyc\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:    9.5s remaining:    8.3s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   10.1s finished


In [157]:
clf 

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [160]:
clf.fit(x_train_tfidf,y_train)
nb_train_predictions =clf.predict(x_train_tfidf)
nb_test_predictions= clf.predict(x_test_tfidf)

In [163]:
accuracy_score(nb_train_predictions, y_train)

0.4690834993035009

In [164]:
accuracy_score(nb_test_predictions, y_test)

0.4248019893166329

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_train, train_probs_lgb_r[:,1])
metrics.auc(fpr, tpr)

In [170]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV

x_train, x_val, y_train, y_val = train_test_split(x_train_tfidf, y_train, test_size = 0.2, random_state=1)

lgb_mdl_rand = lgb.LGBMClassifier(boosting_type = 'gbdt',
                                n_jobs = -1,
                                objective = 'multiclass',
                                num_iterations = 20,
                                metric = 'auc'
                               )

lgb_grid_params_rand = { 
    'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5],
    'estimator__max_depth' : [5, 10, 15, 20, 30],
    'num_leaves': [6,8,12,16,50],
    'random_state' : [1]
}

lgb_gs_rand = RandomizedSearchCV(lgb_mdl_rand, lgb_grid_params_rand,
                                 n_jobs=1, cv=4,
                                 random_state=2)

fit_params={"early_stopping_rounds" : 10,
           "eval_set" : [[x_val, y_val]]}


In [173]:
lgb_gs_rand.fit(x_train, y_train, **fit_params)

C:\Users\Lucyc\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Lucyc\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: y contains previously unseen labels: [8]